# Web Scrapping
---
Este notebook pretende hacer un **bot** para el scrapping de la página 

<a href="https://cis.whoi.edu/science/B/whalesounds/index.cfm"> Watkins Marine Mammal Sound Database



<img src="https://img.icons8.com/dusk/2x/bot--v2.png">

## Modulos
---

In [1]:
#modulos importados
!pip install fake_useragent
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
import numpy as np
import time, re, os, requests

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=4fb736e5fa970ef5eb40ddc3e4fa5b3207db5c0d9945f649b82641122f906d2d
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent
     |████████████████████████████████| 911kB 7.2MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## funciones definidas
---

In [3]:
######FUNCIONES#########
#Timepo de espera con una Distribución de probabilidad lognormal
def distriProb_lognorm():
  pdf = np.random.lognormal(mean=2.0, sigma=0.5, size=(1))
  if pdf<3  or pdf>7:
    pdf=np.random.randint(3,6)
  return float(pdf)

  

## Navegación
---

Aquí se pretende que el bot llegue a cada uno de los links de cada una de las 32 especies de mamaiferos marinos y retorne un dicionario: 
>``{especie : link}``

In [4]:
directorio= os.getcwd()
print(f'Directorio actual es: {directorio}')

#cambiamos el directorio por el de la carpeta que trabajaremos
print('#######PASO 1######')
#en mi caso es: /content/drive/MyDrive/Colab Notebooks/IA y AP/Proyecto/Audios
os.chdir(input('\nColoca la ruta de la carpeta donde van a quedar los audios:'))
directorio= os.getcwd()

#########PASO 1#############
#colocamos la ruta del buscador que vayamos a usar en este caso Chrome
driver =  webdriver.Chrome('chromedriver',options=options)

#colocamos el link de la pagina donde queremos hacer la busqueda
try:
    driver.get('https://cis.whoi.edu/science/B/whalesounds/index.cfm')
except:
    print('Hubo un error de conexion intentando de nuevo...')
    time.sleep(distriProb_lognorm())   
    driver.refresh()
    driver.get('https://cis.whoi.edu/science/B/whalesounds/index.cfm')
    
time.sleep(distriProb_lognorm())
#filtramos las etiquetas de html con la caracteristica que buscamos
soup = BeautifulSoup(driver.page_source, 'html.parser')
html_links = soup.find_all('option', {'value' : True})

whale= {}

for link, idx in zip(html_links, range(len(html_links))):
    name = html_links[idx].text
    m = re.search(' \n\t\t', name)
    if m == None:
        continue
    key = name[:m.start()] + name[m.end():]
    whale[key]= link.get('value')
    
print('\nDiccionario {Especie: link}:', whale)

Directorio actual es: /content
#######PASO 1######

Coloca la ruta de la carpeta donde van a quedar los audios:/content/drive/MyDrive/Colab Notebooks/IA y AP/Proyecto/Audios

Diccionario {Especie: link}: {'Atlantic Spotted Dolphin': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BD15F', 'Bearded Seal': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=CC2A', 'Beluga, White Whale': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BB1A', 'Bottlenose Dolphin': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BD19D', 'Bowhead Whale': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=AA1A', 'Clymene Dolphin': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BD15B', 'Common Dolphin': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BD3B', 'False Killer Whale': 'https://whoicf2.whoi.edu/science/B/whalesounds/bestOf.cfm?code=BE9A', 'Fin, Finback Whale': 'https://whoicf2.whoi.edu/science/B/w

## Descaga
---
Aquí se pretende que el bot vaya al link de cada una de las 32 especies y descargue cada uno de los audios de la misma.

In [5]:
#loop para ir a cada especie y descargar el audio

for link, name, num in zip(whale.values(), whale.keys(), range(len(whale.values()))):  
    driver.get(link)
    time.sleep(distriProb_lognorm())
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    video_links = soup.table.find_all('source', {'src' : True}, src = True)
    print(f'\nBuscando: {name} '+ str(num+1)+'/'+str(len(whale.values())))
    for link, i in zip(video_links, range(len(video_links))):
        ua = UserAgent()
        headers = {'User-Agent': ua.random}

        #solicitamos el codigo html de la pagina
        ruta = 'https://whoicf2.whoi.edu' + link.get('src')
        time.sleep(distriProb_lognorm())
        result = requests.get(ruta, allow_redirects=True, headers=headers)
        print(f'\nDescargando: {ruta[-12:]} como {name+str(i+1)}.mp4')
        open(name+str(i+1)+'.mp4', 'wb').write(result.content)
        print(f'Archivo {name+str(i+1)}.mp4 descargado!')

print('\n¡Listo!--> Base de datos descargada')
    


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Archivo Atlantic Spotted Dolphin52.mp4 descargado!

Descargando: 6102603W.wav como Atlantic Spotted Dolphin53.mp4
Archivo Atlantic Spotted Dolphin53.mp4 descargado!

Descargando: 6102603X.wav como Atlantic Spotted Dolphin54.mp4
Archivo Atlantic Spotted Dolphin54.mp4 descargado!

Descargando: 6102603Z.wav como Atlantic Spotted Dolphin55.mp4
Archivo Atlantic Spotted Dolphin55.mp4 descargado!

Descargando: 6102604J.wav como Atlantic Spotted Dolphin56.mp4
Archivo Atlantic Spotted Dolphin56.mp4 descargado!

Descargando: 6102604K.wav como Atlantic Spotted Dolphin57.mp4
Archivo Atlantic Spotted Dolphin57.mp4 descargado!

Descargando: 6102604L.wav como Atlantic Spotted Dolphin58.mp4
Archivo Atlantic Spotted Dolphin58.mp4 descargado!

Buscando: Bearded Seal 2/32

Descargando: 66038001.wav como Bearded Seal1.mp4
Archivo Bearded Seal1.mp4 descargado!

Descargando: 66038002.wav como Bearded Seal2.mp4
Archivo Bearded Seal2.mp4 desca